방문객 기준의 패턴 분석


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import mysql.connector
import datetime
from gspread_formatting import *
import re
from user_agents import parse as ua_parse
import json
from decimal import Decimal
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

from plotly.subplots import make_subplots


def day_of_week_korean(day_num):
    """
    Convert a day number (1-7) to its Korean representation.
    Note: MySQL's DAYOFWEEK() function considers 1 as Sunday and 7 as Saturday.
    """
    days_korean = {
        1: "일",  # Sunday
        2: "월",  # Monday
        3: "화",  # Tuesday
        4: "수",  # Wednesday
        5: "목",  # Thursday
        6: "금",  # Friday
        7: "토"   # Saturday
    }
    return days_korean.get(day_num, "")


def fetch_visit_data():


    # Connect to MySQL
    connection = mysql.connector.connect(**mysql_config)
    cursor = connection.cursor()

    # Query to fetch customer list
    sql = """
        SELECT
            visit_hour,
            day_of_week,
            COUNT(ip) AS unique_daily_visits
        FROM (
            SELECT
                ip,
                HOUR(CONVERT_TZ(FROM_UNIXTIME(dt), '+00:00', '-09:00')) as visit_hour,
                DATE_FORMAT(CONVERT_TZ(FROM_UNIXTIME(dt), '+00:00', '-09:00'), '%Y-%m-%d') AS visit_date,
                DAYOFWEEK(CONVERT_TZ(FROM_UNIXTIME(dt), '+00:00', '-09:00')) as day_of_week
            FROM wp_slim_stats
            GROUP BY ip, visit_date
        ) AS UniqueDailyVisitors
        GROUP BY visit_hour, day_of_week;
    """
    # Execute the query and fetch the result into a pandas DataFrame
    data = pd.read_sql_query(sql, connection)
    # Close the database connection
    connection.close()
    return data


data = fetch_visit_data()

x_axis_labels = list(range(24))  # Hour labels for x axis

total_visits = data['unique_daily_visits'].sum()


colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']

# Create a bar chart
fig = go.Figure()

for day in range(1, 8):  # Assuming 1 is Sunday and 7 is Saturday as per MySQL's DAYOFWEEK()
    subset = data[data['day_of_week'] == day]
    fig.add_trace(go.Bar(
        x=subset['visit_hour'],
        y=subset['unique_daily_visits'],
        name=day_of_week_korean(day),  # or replace with actual day names
        marker_color=colors[day - 1],
        # text=[f"{hour}시<br>{count} visits<br>({count/total_visits:.2%})" for hour, count in zip(subset['visit_hour'], subset['unique_daily_visits'])],
        text=[f"{count} visits<br>({count/total_visits:.2%})" for hour, count in zip(subset['visit_hour'], subset['unique_daily_visits'])],
        textposition='auto'
    ))


fig.update_layout(
    title="Unique Daily Visits per Hour",
    xaxis=dict(
        title="Hour of the Day",
        tickvals=x_axis_labels,
        ticktext=[str(i) for i in x_axis_labels]
    ),
    yaxis=dict(title="Number of Unique Daily Visits"),
    barmode='stack', height=1200
)

# Show the plot
fig.show()

/tmp/ipykernel_2111613/3738671290.py:66: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [125]:

# Pivot the data to create a matrix
matrix = data.pivot('day_of_week', 'visit_hour', 'unique_daily_visits').fillna(0)

# Calculate the total for percentages
total = matrix.sum().sum()

# Create the hover text and annotations: count, and merged labels
hover_text = []
annotations = []
days_of_week = ['', '월', '화', '수', '목', '금', '토', '일']
for i, day in enumerate(matrix.index):
    hover_row = []
    for j, hour in enumerate(matrix.columns):
        text = f"{matrix.at[day, hour]} visits<br>{days_of_week[i]} {hour}시"
        hover_row.append(text)

        # If count is more than 0, create an annotation
        if matrix.at[day, hour] > 0:
            annotations.append({
                "x": hour,
                "y": day,
                "xref": "x",
                "yref": "y",
                "text": f"{int(hour)}{days_of_week[i+1]}<br>{int(matrix.at[day, hour])}",
                "showarrow": False,
                "font": {
                    "color": "black"
                }
            })
    hover_text.append(hover_row)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=matrix.values,
    x=matrix.columns,
    y=matrix.index,
    colorscale='Viridis',
    hoverinfo='text',
    text=hover_text,
    colorbar=dict(title='Unique Visits')
))

fig.update_layout(
    title="요일별 시간당 Unique visitor 수 matrix",
    xaxis=dict(title="Hour of Day", tickvals=list(range(25)), ticktext=[str(i) for i in range(25)]),
    yaxis=dict(title="Day of Week", tickvals=list(range(8)), ticktext=days_of_week),
    annotations=annotations, height=650, width=2000
)

fig.show()

/tmp/ipykernel_2111613/3134829014.py:2: FutureWarning:

In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.



구독자기준의 패턴 분석


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import mysql.connector
import datetime
from gspread_formatting import *
import re
from user_agents import parse as ua_parse
import json
from decimal import Decimal
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

from plotly.subplots import make_subplots


class DecimalEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return float(obj)
        return super(DecimalEncoder, self).default(obj)


def extract_details(session_token):
    if not isinstance(session_token, (str, bytes)):
        return None
    third_party_pattern = re.compile(r'(BizWebView[ ;]|KAKAOTALK.*?|NAVER\(.*?\))')

    pattern_ip = r's:2:"ip";s:\d+:"(.*?)";'
    pattern_ua = r's:2:"ua";s:\d+:"(.*?)";'

    match_ip = re.search(pattern_ip, session_token)
    match_ua = re.search(pattern_ua, session_token)

    if match_ip and match_ua:
        ip_address = match_ip.group(1)
        user_agent_string = match_ua.group(1)
        user_agent = ua_parse(user_agent_string)

        # Extract third-party info
        third_party_info = third_party_pattern.findall(user_agent_string)
        third_party_info_str = ', '.join(third_party_info) if third_party_info else None

        # Extract browser version, and supplement with third-party info if empty
        browser_version = user_agent.browser.version_string

        return {
            "IP Address": ip_address,
            "Browser Family": user_agent.browser.family,
            "Browser Version": browser_version or '',
            "OS Family": user_agent.os.family,
            "OS Version": user_agent.os.version_string,
            "Device Family": user_agent.device.family,
            "Third Party Info": third_party_info_str or ''
        }
    return None


scope = ["https://spreadsheets.google.com/feeds", 'https://www.'
'apis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

client = gspread.authorize(creds)


def fetch_customer_data():
    # MySQL connection details
#--

    # Connect to MySQL
    connection = mysql.connector.connect(**mysql_config)
    cursor = connection.cursor()

    # Query to fetch customer list
    query = """
    SELECT
        u.ID,
        u.user_login,
        u.user_nicename AS user_nickname,
        u.user_email,
        CONVERT_TZ(u.user_registered, '+00:00', '+09:00') AS user_registered_seoul,
        u.display_name,
        MAX(CASE WHEN um.meta_key = 'session_tokens' THEN um.meta_value END) AS session_tokens,
        MAX(CASE WHEN um.meta_key = 'first_name' THEN um.meta_value END) AS name,
        MAX(CASE WHEN um.meta_key = 'billing_phone' THEN CONCAT("'", um.meta_value) END) AS billing_phone,
        MAX(CASE WHEN um.meta_key = 'shipping_phone' THEN CONCAT("'", um.meta_value) END) AS shipping_phone,
        MAX(CASE
            WHEN um.meta_key = 'gender' THEN
                CASE
                    WHEN um.meta_value IN ('man', 'male') THEN '남'
                    WHEN um.meta_value IN ('woman', 'female') THEN '여'
                    ELSE 'n/a'
                END
            END) AS gender,
        um_birthyear.meta_value AS birthyear,
        MAX(CASE WHEN um.meta_key = 'billing_first_name_kr' THEN um.meta_value END) AS billing_first_name_kr,
        MAX(CASE WHEN um.meta_key = 'billing_address_1' THEN um.meta_value END) AS billing_address_1,
        MAX(CASE WHEN um.meta_key = 'billing_address_2' THEN um.meta_value END) AS billing_address_2,
        MAX(CASE WHEN um.meta_key = 'billing_postcode' THEN um.meta_value END) AS billing_postcode,
        MAX(CASE WHEN um.meta_key = 'shipping_first_name_kr' THEN um.meta_value END) AS shipping_first_name_kr,
        MAX(CASE WHEN um.meta_key = 'shipping_address_1' THEN um.meta_value END) AS shipping_address_1,
        MAX(CASE WHEN um.meta_key = 'shipping_address_2' THEN um.meta_value END) AS shipping_address_2,
        MAX(CASE WHEN um.meta_key = 'last_login_time' THEN um.meta_value END) AS last_login_time,
        sm.first_subscription_time,
        sm.paid_amt,
        sm.subscription_count,
        YEAR(CURDATE()) - um_birthyear.meta_value AS current_age
    FROM
        wp_users AS u
    LEFT JOIN
        wp_usermeta AS um ON u.ID = um.user_id
    LEFT JOIN (
        SELECT
            pm.meta_value AS user_id,
            MIN(CONVERT_TZ(p.post_date, '+00:00', '+09:00')) AS first_subscription_time,
            SUM(CAST(pm_total.meta_value AS UNSIGNED)) AS paid_amt,
            COUNT(oi.order_item_id) AS subscription_count
        FROM
            wp_posts p
        LEFT JOIN
            wp_postmeta pm ON p.ID = pm.post_id AND pm.meta_key = '_customer_user'
        LEFT JOIN
            wp_postmeta pm_total ON p.ID = pm_total.post_id AND pm_total.meta_key = '_order_total'
        LEFT JOIN
            wp_woocommerce_order_items oi ON p.ID = oi.order_id
        WHERE
            p.post_type = 'shop_subscription' AND p.post_status = 'wc-active'
        GROUP BY
            pm.meta_value
    ) AS sm ON u.ID = CAST(sm.user_id AS UNSIGNED)
    LEFT JOIN wp_usermeta AS um_birthyear ON u.ID = um_birthyear.user_id AND um_birthyear.meta_key = 'birthyear'
    WHERE
        user_email NOT LIKE '%cleanbedding.kr'
        AND user_email NOT LIKE '%is-global.sg%'
        AND user_email NOT LIKE '%test%'
        AND display_name NOT IN  ('클린베딩 담당자','정종필','정지혜_테스트계정',
        'TestUser',
        'Confront your fears',
        'test@codem.com',
        'test1@codem.com',
        'test@0509.com',
        'test001',
        'test002',
        '테스트1',
        'Andrew Benedictus',
        'codemtest',
        'test',
        'H.jeong',
        'koa',
        'support@elementor.com',
        '고객관리자',
    GROUP BY
        u.user_registered, u.ID, sm.first_subscription_time, sm.paid_amt, sm.subscription_count
    ORDER BY
        u.ID;
    """

    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows[:10]:
        session_token_index = [col[0] for col in cursor.description].index('session_tokens')
        # print("Session token:", row[session_token_index])

    # Extract the OS information from the session_tokens

    def extract_os_from_session(session_token):
        ua_match = re.search(r'"ua";s:\d+:"(.*?)";', session_token)
        if ua_match:
            return extract_os(ua_match.group(1))
        return "Unknown"

    headers = [col[0] for col in cursor.description if col[0] != 'session_tokens'] + ["IP Address",
                                                                                      "Browser Family", "Browser Version", "OS Family", "OS Version", "Device Family", "Third Party Info"]
    session_token_index = [col[0] for col in cursor.description].index('session_tokens')

    for i, r in enumerate(rows):
        print(r)
        if i > 5:
            break
    print()
    print()

    # Step 1: Replace phone numbers
    formatted_rows_step1 = [
        [
            item.replace("'8210", "010").replace("'+82 ", "0").replace("'", "") if isinstance(item, str) else
            str(item) if isinstance(item, datetime.datetime) else item
            for index, item in enumerate(row) if index != session_token_index
        ]
        for row in rows
    ]
    for i, r in enumerate(formatted_rows_step1):
        print(r)
        if i > 5:
            break

    for i, r in enumerate(formatted_rows_step1):
        print(r)
        if i > 5:
            break

    # Step 3: Extract details
    formatted_rows_final = [
        row + list(extract_details(original_row[session_token_index]).values()) if extract_details(original_row[session_token_index]) else row
        for row, original_row in zip(formatted_rows_step1, rows)
    ]
    print(headers)
    for i, r in enumerate(formatted_rows_final):
        print(r)
        if i > 5:
            break
    data_to_insert = [headers] + formatted_rows_final

    #
    # 기족 고객 여부 추가
    #

    # Fetch the old customer data starting from the 6th row
    old_data = old_sheet.get_all_values()[5:]  # Starting from 6th row (0-based index)
    old_data_headers = old_data[0]
    old_data_records = [dict(zip(old_data_headers, row)) for row in old_data[1:]]

    # Step 3: Check each entry in the data_to_insert list against the old customer list
    existing_customers = set([(entry["이름"], entry["휴대폰 번호"].replace("-", "")) for entry in old_data_records])
    new_customer_statuses = []
    for entry in data_to_insert[1:]:
        phone = entry[headers.index("billing_phone")] if entry[headers.index("billing_phone")] else entry[headers.index("shipping_phone")]
        if phone is not None:
            phone = phone.replace("-", "")
        if (entry[headers.index("name")], phone) in existing_customers:
            new_customer_statuses.append("기존")
        else:
            new_customer_statuses.append("신규")
    # Step 4: Insert '기존고객' column in data_to_insert list
    headers.insert(1, '기존고객')
    for index, row in enumerate(data_to_insert[1:], start=1):
        row.insert(1, new_customer_statuses[index - 1])

    # Close MySQL connection
    cursor.close()
    connection.close()
    df = pd.DataFrame(data_to_insert[1:], columns=data_to_insert[0])
    return df


df = fetch_customer_data()


# 신규 사이트의 신규 회원 수
total_new_site_members_today = df.shape[0]
df['user_registered_seoul'] = pd.to_datetime(df['user_registered_seoul'])

# 오늘의 신규 회원 가입자 수
new_members_today = df[df['기존고객'] == '신규'].shape[0]

# 구 사이트에서 이전한 회원 수
migrated_members = df[df['기존고객'] == '기존'].shape[0]

# 'user_registered_seoul'의 날짜만 추출하여 새로운 컬럼에 저장
df['registration_date'] = df['user_registered_seoul'].dt.date

# 날짜별로 신규 회원과 구사이트 회원의 가입 수치 계산
daily_new = df[df['기존고객'] == '신규'].groupby('registration_date').size()
daily_migrated = df[df['기존고객'] == '기존'].groupby('registration_date').size()

# 누락된 날짜에 대한 데이터 추가 (0으로 채우기)
all_dates = pd.date_range(start=df['registration_date'].min(), end=df['registration_date'].max(), freq='D')
daily_new = daily_new.reindex(all_dates, fill_value=0)
daily_migrated = daily_migrated.reindex(all_dates, fill_value=0)

# 일별 누적 가입 수치 계산
cumulative_new = daily_new.cumsum()
cumulative_migrated = daily_migrated.cumsum()

# df[~df['subscription_count'].isna()]

In [117]:
def day_of_week_korean_pandas(day_num):
    """
    Convert a day number (1-7) to its Korean representation.
    Note: MySQL's DAYOFWEEK() function considers 1 as Sunday and 7 as Saturday.
    """
    days_korean = {
        6: "일",  # Sunday
        0: "월",  # Monday
        1: "화",  # Tuesday
        2: "수",  # Wednesday
        3: "목",  # Thursday
        4: "금",  # Friday
        5: "토"   # Saturday
    }
    return days_korean.get(day_num, "")


data = df[df['기존고객'] == '신규']
# data = df
x_axis_labels = list(range(24))  # Hour labels for x axis

# total_visits = new_members_today

total_subscribed = data[data['subscription_count'] > 0].shape[0]
# total_non_subscribed = data.shape[0] - total_subscribed
total_visits = total_subscribed
data['first_subscription_time'] = pd.to_datetime(data['first_subscription_time'])
data['day_of_week'] = data['first_subscription_time'].dt.dayofweek
data['visit_hour'] = data['first_subscription_time'].dt.hour
data = data.groupby(['day_of_week', 'visit_hour']).size().reset_index(name='unique_daily_visits')


colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']

# Create a bar chart
fig = go.Figure()

# for day in range(1, 8):  # Assuming 1 is Sunday and 7 is Saturday as per MySQL's DAYOFWEEK()
for day in range(1, 7):  # Assuming 1 is Sunday and 7 is Saturday as per MySQL's DAYOFWEEK()
    subset = data[data['day_of_week'] == day]
    fig.add_trace(go.Bar(
        x=subset['visit_hour'],
        y=subset['unique_daily_visits'],
        name=day_of_week_korean_pandas(day),  # or replace with actual day names
        marker_color=colors[day - 1],
        # text=[f"{hour}시<br>{count} visits<br>({count/total_visits:.2%})" for hour, count in zip(subset['visit_hour'], subset['unique_daily_visits'])],
        text=[f"{count} visits<br>({count/total_visits:.2%})" for hour, count in zip(subset['visit_hour'], subset['unique_daily_visits'])],
        textposition='auto'
    ))


fig.update_layout(
    title="요일별 시간당 '신규' 구독자 수",
    xaxis=dict(
        title="Hour of the Day",
        tickvals=x_axis_labels,
        ticktext=[str(i) for i in x_axis_labels]
    ),
    yaxis=dict(title="Number of Unique Daily Visits"),
    barmode='stack', height=1200
)

# Show the plot
fig.show()

/tmp/ipykernel_2111613/2245814822.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2111613/2245814822.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2111613/2245814822.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [118]:
import plotly.graph_objects as go
import numpy as np


# Pivot the data to create a matrix
matrix = data.pivot('day_of_week', 'visit_hour', 'unique_daily_visits').fillna(0)

# Calculate the total for percentages
total = matrix.sum().sum()

# Create the hover text and annotations: count, and merged labels
hover_text = []
annotations = []
days_of_week = ['월', '화', '수', '목', '금', '토', '일']
for i, day in enumerate(matrix.index):
    hover_row = []
    for j, hour in enumerate(matrix.columns):
        text = f"{matrix.at[day, hour]} visits<br>{days_of_week[i]} {hour}시"
        hover_row.append(text)

        # If count is more than 0, create an annotation
        if matrix.at[day, hour] > 0:
            annotations.append({
                "x": hour,
                "y": day,
                "xref": "x",
                "yref": "y",
                "text": f"{int(hour)}{days_of_week[i]}<br>{int(matrix.at[day, hour])}",
                "showarrow": False,
                "font": {
                    "color": "black"
                }
            })
    hover_text.append(hover_row)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=matrix.values,
    x=matrix.columns,
    y=matrix.index,
    colorscale='Viridis',
    hoverinfo='text',
    text=hover_text,
    colorbar=dict(title='Unique Visits')
))

fig.update_layout(
    title="요일별 시간당 '신규' 구독자 수 matrix",
    xaxis=dict(title="Hour of Day", tickvals=list(range(25)), ticktext=[str(i) for i in range(25)]),
    yaxis=dict(title="Day of Week", tickvals=list(range(8)), ticktext=days_of_week),
    annotations=annotations, height=650, width=2000
)

fig.show()

/tmp/ipykernel_2111613/1465318856.py:6: FutureWarning:

In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.

